# Deepfake detection using GAN

In [1]:
# Imports
import numpy as np
import random
import torch
import glob
from imutils.video import FileVideoStream
from tqdm.notebook import tqdm
import cv2
from PIL import Image

Controlling seeds to control reproducability.

In [2]:
if False:
    seed = 0
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [3]:
filenames = glob.glob('sample_of_sample/*.mp4')
images = []
# Number of frames to sample (evenly spaced) from each video
n_frames = 10
for filename in tqdm(filenames):
    print(filename)
    v_cap = FileVideoStream(filename).start()
    v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))
    sample = np.linspace(0, v_len - 1, n_frames).round().astype(int)
    for j in range(v_len):
        frame = v_cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if j in sample:
           images.append(Image.fromarray(frame))
    v_cap.release()

  0%|          | 0/1 [00:00<?, ?it/s]

sample_of_sample\eggbjzxnmg.mp4


AttributeError: 'FileVideoStream' object has no attribute 'release'